In [ ]:
from datasets import load_dataset
from transformers import AutoTokenizer,AutoModelForTokenClassification
from transformers import DataCollatorForTokenClassification
from transformers import Trainer,TrainingArguments,TrainerCallback
import numpy as np
import evaluate
from torch.utils.tensorboard import SummaryWriter

In [ ]:
dataset = load_dataset("PassbyGrocer/msra-ner",trust_remote_code=True)
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

In [ ]:
print(dataset["train"].features["ner_tags"])
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased",
                                                        num_labels=len(dataset['train'].features['ner_tags'].feature.names))
id2label = {i:dataset['train'].features['ner_tags'].feature.names[i] for i in range(len(dataset['train'].features['ner_tags'].feature.names))}
model.config.id2label = id2label
model.config.label2id = {v: k for k, v in id2label.items()}

In [ ]:
label_list = dataset["train"].features["ner_tags"].feature.names
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, padding=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        label_ids = [-100 if word_id is None else label[word_id] for word_id in word_ids]
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

tokenized_datasets = dataset.map(tokenize_and_align_labels, batched=True)
data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
metric = evaluate.load("seqeval")
writer = SummaryWriter(log_dir="./logs")
label_list = dataset["train"].features["ner_tags"].feature.names  # 获取标签名称列表

def compute_metrics(p):
    # 获取 logits，并将它们转换为预测的标签索引
    predictions = np.argmax(p.predictions, axis=2)
    references = p.label_ids

    # 转换为字符串标签，并过滤掉 -100
    true_predictions = [
        [label_list[pred] for (pred, label) in zip(prediction, reference) if label != -100]
        for prediction, reference in zip(predictions, references)
    ]
    true_labels = [
        [label_list[label] for (pred, label) in zip(prediction, reference) if label != -100]
        for prediction, reference in zip(predictions, references)
    ]

    # 计算 seqeval 指标
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [ ]:
class TensorBoardCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if logs is None:
            return
        # 将每次日志的损失值记录到 TensorBoard
        for k, v in logs.items():
            if "loss" in k:
                writer.add_scalar(f"Loss/{k}", v, global_step=state.global_step)

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",                # 指定日志目录
    logging_strategy="steps",            # 设置日志记录策略（这里按步记录）
    logging_steps=10,                    # 每 10 步记录一次日志
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=1,
    weight_decay=0.01,
    fp16=True,
    push_to_hub=True,
    hub_model_id="PassbyGrocer/bert-ner-msra"
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[TensorBoardCallback()]
)

trainer.train()

In [ ]:
test_results = trainer.predict(tokenized_datasets["test"])

# 输出测试结果
print("测试集上的结果:", test_results.metrics)

In [ ]:
trainer.push_to_hub()

In [ ]:
!pip install transformers datasets evaluate seqeval tensorboard "numpy<2" transformers[torch] "accelerate>=0.26.0" pytorch-crf